In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from keras.applications.vgg16 import preprocess_input
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [2]:
data = pd.read_csv('./run4.csv')     # reading the csv file
data.head()
X = [ ]     # creating an empty array
for img_name in data.IMG_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

/tmp/ipykernel_49/423475003.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)    # converting list to array


In [3]:
y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

In [4]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

In [5]:
X = preprocess_input(X)      # preprocessing the input data

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

In [8]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((105, 7, 7, 512), (46, 7, 7, 512))

In [9]:
X_train = X_train.reshape(105, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(46, 7*7*512)

In [10]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [11]:
#i. building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [13]:
# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# iii. Training the model
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.5770 - accuracy: 0.5714 - val_loss: 0.2571 - val_accuracy: 0.9565
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.1069 - accuracy: 0.9905 - val_loss: 0.2004 - val_accuracy: 0.8913
Epoch 3/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0444 - accuracy: 1.0000 - val_loss: 0.1145 - val_accuracy: 0.9565
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0136 - accuracy: 1.0000 - val_loss: 0.0977 - val_accuracy: 0.9565
Epoch 5/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.0965 - val_accuracy: 0.9565
Epoch 6/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0938 - val_accuracy: 0.9565
Epoch 7/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0893 - val_accuracy: 0.9565
Epoch 8/100
4/4 [===

In [15]:
test = pd.read_csv('./test.csv')
test_image = []
for img_name in test.IMG_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

/tmp/ipykernel_49/4029798551.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_img = np.array(test_image)


In [16]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [17]:
# preprocessing the images
test_image1 = preprocess_input(test_image)
test_image1.shape
# extracting features from the images using pretrained model
test_image2 = base_model.predict(test_image1)

# converting the images to 1-D form
test_image3 = test_image2.reshape(104, 7*7*512)

# zero centered images
test_image4 = test_image3/test_image3.max()

In [18]:
predictions = model.predict(test_image4)

In [19]:
print(predictions)

[[9.56350029e-01 4.36499864e-02]
 [9.02720630e-01 9.72793698e-02]
 [9.52453673e-01 4.75463532e-02]
 [9.82324839e-01 1.76751856e-02]
 [9.91861582e-01 8.13835301e-03]
 [9.89492655e-01 1.05073387e-02]
 [9.74745512e-01 2.52545029e-02]
 [7.11730599e-01 2.88269401e-01]
 [9.05377507e-01 9.46225077e-02]
 [9.64600086e-01 3.53998654e-02]
 [9.91194606e-01 8.80534761e-03]
 [9.17770803e-01 8.22291449e-02]
 [9.93250668e-01 6.74927793e-03]
 [9.87526357e-01 1.24736438e-02]
 [3.14353168e-01 6.85646832e-01]
 [6.23726249e-01 3.76273751e-01]
 [9.79976058e-01 2.00239476e-02]
 [9.97591019e-01 2.40895175e-03]
 [9.54091251e-01 4.59087603e-02]
 [8.33266497e-01 1.66733488e-01]
 [9.57065463e-01 4.29345332e-02]
 [8.20798159e-01 1.79201812e-01]
 [9.93723691e-01 6.27630204e-03]
 [9.90315735e-01 9.68432892e-03]
 [9.96444166e-01 3.55582568e-03]
 [9.92614985e-01 7.38505740e-03]
 [9.97116804e-01 2.88321869e-03]
 [9.77725387e-01 2.22746823e-02]
 [9.88961041e-01 1.10388882e-02]
 [7.99439728e-01 2.00560287e-01]
 [6.546786

In [20]:
i =0
for r in predictions:
    if r[0] > r[1] :
      print("Pluto is anxious--> " + str(r[0])+" " + test.IMG_ID[i])
    if r[0] < r[1] :
      print("Pluto is NOT anxious--> "+ str(r[1])+" " + test.IMG_ID[i])
    i = i +1

Pluto is anxious--> 0.95635 ./data/testCases/frame100.jpg
Pluto is anxious--> 0.90272063 ./data/testCases/frame107.jpg
Pluto is anxious--> 0.9524537 ./data/testCases/frame115.jpg
Pluto is anxious--> 0.98232484 ./data/testCases/frame124.jpg
Pluto is anxious--> 0.9918616 ./data/testCases/frame127.jpg
Pluto is anxious--> 0.98949265 ./data/testCases/frame132.jpg
Pluto is anxious--> 0.9747455 ./data/testCases/frame149.jpg
Pluto is anxious--> 0.7117306 ./data/testCases/frame155.jpg
Pluto is anxious--> 0.9053775 ./data/testCases/frame168.jpg
Pluto is anxious--> 0.9646001 ./data/testCases/frame181.jpg
Pluto is anxious--> 0.9911946 ./data/testCases/frame186.jpg
Pluto is anxious--> 0.9177708 ./data/testCases/frame193.jpg
Pluto is anxious--> 0.99325067 ./data/testCases/frame207.jpg
Pluto is anxious--> 0.98752636 ./data/testCases/frame211.jpg
Pluto is NOT anxious--> 0.68564683 ./data/testCases/frame218.jpg
Pluto is anxious--> 0.62372625 ./data/testCases/frame222.jpg
Pluto is anxious--> 0.97997606 